# TP3 - Problema 1

### Grupo 11

Nelson Almeida a95652
<br>
Nuno Costa a97610


Pretende-se construir uma implementação simplificada do algoritmo “model checking” orientado aos interpolantes seguindo a estrutura apresentada nos apontamentos onde no passo $$(n,m)\,$$na impossibilidade de encontrar um interpolante invariante se dá ao utilizador a possibilidade de incrementar um dos índices $$n$$ e $$m$$ à sua escolha.
Pretende-se aplicar este algoritmo ao problema da da multiplicação de inteiros positivos em `BitVec`  (apresentado no TP2).


In [38]:
from pysmt.shortcuts import *
from pysmt.typing import INT

import itertools 

In [39]:
def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),INT)
    return state

In [40]:
def declare(i, n):
    state = {}
    
    # definir variáveis
    state['node'] = Symbol('node' + str(i), INT)
    state['x'] = Symbol('x' + str(i), BVType(n))
    state['y'] = Symbol('y' + str(i), BVType(n))
    state['z'] = Symbol('z' + str(i), BVType(n))
    
    return state

In [41]:
def init(state,a,b):
    return And(
        Equals(state['node'], Int(0)),
        Equals(state['x'], Int(a)),
        Equals(state['y'], Int(b)),
        Equals(state['z'], Int(0))
    )

In [42]:
def even(atual):
    return Equals(Times(Div(atual['y'],Int(2)),Int(2)), atual['y'] )


In [43]:
def trans(atual, prox):
    # 𝑛𝑜𝑑𝑒=0 ∧ ( 𝑦≠0 ∧ 𝑒𝑣𝑒𝑛(𝑦) ) ∧ 𝑛𝑜𝑑𝑒′=1 ∧ 𝑥′=𝑥 ∧ 𝑦′=𝑦 ∧ 𝑧′=𝑧
    t0 = And(
        Equals(atual['node'], Int(0)),
        NotEquals(atual['y'], Int(0)),
        even(atual),
        Equals(prox['node'], Int(1)),
        Equals(prox['x'], atual['x']),
        Equals(prox['y'], atual['y']),
        Equals(prox['z'], atual['z'])
    )
    
    # 𝑛𝑜𝑑𝑒=0 ∧ 𝑦=0 ∧ 𝑛𝑜d𝑒′=2 ∧ 𝑥′=𝑥 ∧ 𝑦′=𝑦 ∧ 𝑧′=𝑧
    t1 = And(
        Equals(atual['node'], Int(0)),
        Equals(atual['y'], Int(0)),
        Equals(prox['node'], Int(2)),
        Equals(prox['x'], atual['x']),
        Equals(prox['y'], atual['y']),
        Equals(prox['z'], atual['z'])
    )
    
    # 𝑛𝑜𝑑𝑒=0 ∧ ( 𝑦≠0 ∧ 𝑜𝑑𝑑(𝑦) ) ∧ 𝑛𝑜𝑑𝑒′=3 ∧ 𝑥′=𝑥 ∧ 𝑦′=𝑦 ∧ 𝑧′=𝑧
    t2 = And(
        Equals(atual['node'], Int(0)),
        Not(Equals(atual['y'], Int(0))),
        Not(even(atual)),
        Equals(prox['node'], Int(3)),
        Equals(prox['x'], atual['x']),
        Equals(prox['y'], atual['y']),
        Equals(prox['z'], atual['z'])
    )
    
    # 𝑛𝑜𝑑𝑒=1 ∧ 𝑛𝑜𝑑𝑒′=0 ∧ 𝑥'=2∗𝑥 ∧ 𝑦'=𝑦/2 ∧ 𝑧'=𝑧
    t3 = And(
        Equals(atual['node'], Int(1)),
        Equals(prox['node'], Int(0)),
        Equals(prox['x'], 2*atual['x']),
        Equals(prox['y'], atual['y']/2),
        Equals(prox['z'], atual['z'])
    )

    # 𝑛𝑜𝑑𝑒=3 ∧ 𝑛𝑜𝑑𝑒′=0 ∧ 𝑥′=𝑥 ∧ 𝑦′=𝑦−1 ∧ 𝑧′=𝑧+𝑥
    t4 = And(
        Equals(atual['node'], Int(3)),
        Equals(prox['node'], Int(0)),
        Equals(prox['x'], atual['x']),
        Equals(prox['y'], atual['y']-Int(1)),
        Equals(prox['z'], atual['z']+atual['x'])
    )
      
    # 𝑛𝑜𝑑𝑒=1 ∧ 𝑛𝑜𝑑𝑒′=4 ∧ 𝑜𝑣𝑒𝑟𝑓𝑙𝑜𝑤 ∧ 𝑥′=𝑥 ∧ 𝑦′=𝑦 ∧ 𝑧′=z
    t5 = And(
        Equals(atual['node'], Int(1)),
        Equals(prox['node'], Int(4)),
        Equals(prox['x'], atual['x']),
        Equals(prox['y'], atual['y']),
        Equals(prox['z'], atual['z'])
    )
    
    # 𝑛𝑜𝑑𝑒=3 ∧ 𝑛𝑜𝑑𝑒′=5 ∧ 𝑜𝑣𝑒𝑟𝑓𝑙𝑜𝑤 ∧ 𝑥′=𝑥 ∧ 𝑦′=𝑦 ∧ 𝑧′=𝑧
    t6 = And(
        Equals(atual['node'], Int(3)),
        Equals(prox['node'], Int(5)),
        Equals(prox['x'], atual['x']),
        Equals(prox['y'], atual['y']),
        Equals(prox['z'], atual['z'])
    )
    
    # 𝑛𝑜𝑑𝑒=4 ∧ 𝑛𝑜𝑑𝑒′=4 ∧ 𝑥′=𝑥 ∧ 𝑦′=𝑦 ∧ 𝑧′=𝑧
    t7 = And(
        Equals(atual['node'], Int(4)),
        Equals(prox['node'], Int(4)),
        Equals(prox['x'], atual['x']),
        Equals(prox['y'], atual['y']),
        Equals(prox['z'], atual['z'])
    )
    
    # 𝑛𝑜𝑑𝑒=5 ∧ 𝑛𝑜𝑑𝑒′=5 ∧ 𝑥′=𝑥 ∧ 𝑦′=𝑦 ∧ 𝑧′=𝑧
    t8 = And(
        Equals(atual['node'], Int(5)),
        Equals(prox['node'], Int(5)),
        Equals(prox['x'], atual['x']),
        Equals(prox['y'], atual['y']),
        Equals(prox['z'], atual['z'])
    )
    
    # 𝑛𝑜𝑑𝑒=2 ∧ 𝑛𝑜𝑑𝑒′=2 ∧ 𝑥′=𝑥 ∧ 𝑦′=𝑦 ∧ 𝑧′=𝑧
    t9 = And(
        Equals(atual['node'], Int(2)),
        Equals(prox['node'], Int(2)),
        Equals(prox['x'], atual['x']),
        Equals(prox['y'], atual['y']),
        Equals(prox['z'], atual['z'])
    )
    
    return Or(t0, t1, t2, t3, t4, t5, t6, t7, t8, t9)

In [80]:
def error(i,n):
        return Or(GE(i["x"], Int( pow(2,n))), GE(Plus(i['x'],i['z']), Int(pow(2,n))))
    

In [81]:
def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i), INT)
    return state

In [82]:
def genTrace(vars, init, trans,x,y,n):
    with Solver(name="z3") as s:

        X=[genState(vars,"X",i) for i in range(n+1)]
        I = init(X[0],x,y)
        Tks = [trans(X[i],X[i+1]) for i in range(n)]

        if s.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))

In [83]:
x=5
y=2
n=50
genTrace(['node','x','y','z'],init,trans,x,y,n)

Estado: 0
           node = 0
           x = 5
           y = 2
           z = 0
Estado: 1
           node = 1
           x = 5
           y = 2
           z = 0
Estado: 2
           node = 0
           x = 10
           y = 1
           z = 0
Estado: 3
           node = 3
           x = 10
           y = 1
           z = 0
Estado: 4
           node = 0
           x = 10
           y = 0
           z = 10
Estado: 5
           node = 2
           x = 10
           y = 0
           z = 10
Estado: 6
           node = 2
           x = 10
           y = 0
           z = 10
Estado: 7
           node = 2
           x = 10
           y = 0
           z = 10
Estado: 8
           node = 2
           x = 10
           y = 0
           z = 10
Estado: 9
           node = 2
           x = 10
           y = 0
           z = 10
Estado: 10
           node = 2
           x = 10
           y = 0
           z = 10
Estado: 11
           node = 2
           x = 10
           y = 0
           z = 10
Estado: 

### BMC

In [84]:
def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

# trans(u,v)^-1 = trans(v,u) 
def invert(trans):
    return (lambda u, v : trans(v,u))

In [85]:
def model_checking(vars,init,trans,N,M,a1,b1):
    with Solver(name="z3") as s:
        
        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars,'X',i) for i in range(N+1)]
        Y = [genState(vars,'Y',i) for i in range(M+1)]
        
        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1]) 
        
        for (n,m) in order:
            I = init (X[0],a1,b1)
            Tn = And ([trans(X[i], X[i+1]) for i in range(n)])
            Rn = And (I, Tn)
            
            E = error(Y[0],N)
            Bm = And ([invert(trans)(Y[i], Y[i+1]) for i in range(m)])
            Um = And (E, Bm)
            
            Vnm = And (Rn, same(X[n], Y[m]), Um) 
            
            if s.solve([Vnm]):
                print("unsafe")
                return 
            
            else: # Vnm insatisfazível
                C = binary_interpolant (And (Rn, same(X[n], Y[m])), Um)
                
                if C is None:
                    print("interpolant None")
                    continue
                    
                C0 = rename(C, X[0])
                C1 = rename(C, X[1])
                T = And(X[0], X[1])
                
                if not s.solve([C0, T, Not(C1)]): # C é invariante de T
                    print("safe")
                    return
                
                else: # tenta gerar o majorante S
                    S = rename (C, X[n])
                    while True:
                        A = And(S, trans, X[n], Y[m])
                        if s. solve(A, Um):
                            print("Não é possivel encontrar um majorante")
                            break
                        
                        else:
                            Cnew = binary_interpolant(A, Um)
                            Cn = rename(Cnew, X[n])
                            
                            if s.solve([Cn, Not(S)]): # se Cn->S não é tautologia
                                S = Or(S, Cn)
                                
                            else: # S foi encontado
                                print("safe")
                                return 
            
              
        print("unknown")                   
        

model_checking(['x','y','z','node'], init, trans, 1, 1,5,3)            

unsafe
